## Paczkowanie kodu
Gdy zachodzi potrzeba ponownego użycia pewnego już istniejącego kawałka logiki w innej aplikacji, dobrym pomysłem jest zapakowanie go jako biblioteki i umieszczenie w firmowym repozytorium artefaktów - w przypadku Pythona najlepiej zgodnym z `PyPI`. W [centralnym repozytorium jest paczka](https://pypi.org/project/pypiserver/), którą można odpalić na swoim serwerze i przy jej pomocy stworzyć wewnętrzne repozytorium zgodne ze standardowymi narzędziami w Pythonie - np.: `pip`-em. Na tych warsztatach nie będziemy stawiać takiego serwera, ale pokażemy narzędzia i proces budowania paczek z własnego kodu. Ponieważ główna funkcjonalność naszego narzędzia jest już gotowa, możemy ją spakować i udostępnić w centralnym repozytorium - `pypi.org`.

### Setuptools
`Setuptools` to de facto standardowe narzędzie budowania paczek w ekosystemie Pythona. Nie jest pozbawione wad (zwłaszcza starsze wersje, konfigurowane wyłącznie w kodzie, co utrudniało automatyzację workflowów), jednak zdecydowanie najpopularniejsze. Aby z niego skorzystać musimy je mieć zainstalowane:
```
pip install setuptools
```
choć jeśli używamy `virtualenv`, to zostanie zainstalowane domyślnie.

#### Konfiguracja
Historycznie najbardziej podstawowy sposób konfiguracji `setuptools` polega na stworzeniu pliku o nazwie `setup.py`, zawierającego:

In [ ]:
from setuptools import setup, find_packages

setup(
    name="my-library",
    author="Krzysztof Wiśniewski",
    author_email="krzysztof.wisniewski@bottega.com.pl",
    description="A simple utility for printing foo",
    version="0.0.1",
    packages=find_packages(),
    url="https://github.com/k-wisniewski/my-library",
    install_requires=["flask>=2.0.0"],
    entry_points={
        'console_scripts': [
            'print-foo = foo_printer.main:main'
        ]
    }
)

Oprócz metadanych takich jak nazwa biblioteki, autora, emaila, opisu, wersji etc., wywołanie funkcji setup zawiera również listę pakietów wchodzących w skład paczki (w tym wypadku znalezionych automatyczni w aktualnym katalogu funkcją `find_packages`), zależności czasu budowania i czasu wykonania, a nawet informacje o tym jak stworzyć wykonywalny skrypt, niezależny od systemu operacyjnego (służy do tego argument `entry_points` i jego opcja `console_scripts`).

#### Konfiguracja deklaratywna - zalecana*!
Mimo że historycznie `setuptools` był konfigurowany w pliku `setup.py`, obecnie od tego sposobu się już odchodzi. Wciąż jest duża szansa spotkać się z `setup.py` w istniejących projektach, jednak nie powinno się raczej tworzyć nowych projektów, które na tym sposobie polegają. Powodem są następujące problemy:
- trudna automatyzacja, wymagająca parsowania Pythona
- duża dowolność w sposobie użycia narzędzia, co utrudnia budowanie na jego podstawie bardziej wysokopoziomowych systemów budowania i zmniejsza przenośność

Aby im zaradzić powstał format deklaratywnej konfiguracji, zapisywanej w pliku `setup.cfg`:
```
[metadata]
name = my_package
version = attr: src.VERSION
description = My package description
long_description = file: README.rst, CHANGELOG.rst, LICENSE.rst
keywords = one, two
license = BSD 3-Clause License
classifiers =
    Framework :: Django
    License :: OSI Approved :: BSD License
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.5

[options]
include_package_data = True
packages = find:
scripts =
    bin/first.py
    bin/second.py
install_requires =
    requests
    importlib; python_version == "3.6"

[options.entry_points]
console_scripts =
    executable-name = package.module:function

[options.packages.find]
exclude =
    src.subpackage1
    src.subpackage2
    
    
```

Alternatywnie, korzystając z nowych `setuptools` (od wersji `61.0.0`), można tą konfigurację umieścić w pliku `pyproject.toml`, wskazując `setuptools` jako preferowany system budowania paczek:
```
[build-system]
requires = ["setuptools", "setuptools-scm"]
build-backend = "setuptools.build_meta"

[project]
name = "my_package"
authors = [
    {name = "Josiah Carberry", email = "josiah_carberry@brown.edu"},
]
description = "My package description"
readme = "README.rst"
requires-python = ">=3.7"
keywords = ["one", "two"]
license = {text = "BSD-3-Clause"}
classifiers = [
    "Framework :: Django",
    "Programming Language :: Python :: 3",
]
dependencies = [
    "requests",
    'importlib-metadata; python_version<"3.8"',
]
dynamic = ["version"]

[project.optional-dependencies]
pdf = ["ReportLab>=1.2", "RXP"]
rest = ["docutils>=0.3", "pack ==1.1, ==1.3"]

[project.scripts]
my-script = "my_package.module:function"
```

### Moduł build
W celu uproszczenia i modularyzacji systemów budowania, społeczność Pythona przyjęła `PEP-517` i `PEP-518`. Od tamtej pory odchodzi się od bezpośredniego korzystania z `setuptools` przez użytkowników. Zamiast tego cała konfiguracja docelowo ma znajdować się w pliku `pyproject.toml`, który w założeniu ma zawierać wszystkie konieczne konfiguracje projektu i narzędzi w nim używanych. Do niedawna `setuptools` nie wspierało w pełni tego sposobu konfiguracji, dlatego wciąż często używa sie pliku `setup.cfg`, jednak jest on już zastępowany przez odpowiednie sekcje `pyproject.toml`. Obecnie najnowszy sposób budowania paczek używa pakietu `build`, który wywołuje pod spodem narzędzia budowania zapisane w sekcji `build-system` pliku `pyproject.toml`. Aby z niego skorzystać, trzeba go zainstalować poleceniem:
```
pip install build[virtualenv]
```
`[virtualenv]` przy nazwie pakietu wybierze wariant wspierający virtualenv, co jest konieczne jeśli nasze lokalne środowisko powstało przyy użyciu pakietu `virtualenv` a nie `venv`.
Po instalacji, jeśli posiadamy już `pyproject.toml` i `setup.cfg` możemy zbudować paczkę poleceniem:
```
pyproject-build
```

W rezultacie gotowa do uploadu paczka znajdzie się w katalogu `dist/`. Domyślnie zostanie zbudowana zarówno dystryybucja zawierająca źródła, jak i zbudowany pakiet `wheel`. Można go bezpośrednio instalować poleceniem:
```
pip install dist/<nazwa_pakietu>-<wersja>.whl
```



### Twine
Twine służy do wgrywania paczek na PyPI. Ponieważ tworzone na zajęciach paczki nie są wartościowe z punktu widzenia całego ekosystemu Pythona, wgramy je na specjalną testową instancję PyPI. Aby to zrobić należy najpierw zainstalować `twine`:
```
pip install twine
```
Następnie należy uzyskać token autoryzujący nalezy zalogować się na swoje konto na [TestPyPI](https://test.pypi.org/), wejść w `Account Settings`, a następnie dodać nowy token klikając `Add API Token`. Po podaniu nazwy nowy token zostanie wygenerowany - możemy go skopiować i zapisać w keyringu - managerze sekretów, zainstalowanym razem z twine:
```
keyring set https://test.pypi.org/legacy/ __token__
```
Powyższa komenda poprosi o podanie hasła do zapisania - wklejamy wówczas skopiownay token.

Ostatnim krokiem jest wgranie paczki - na przykład tak:
```
twine upload --username __token__ --repository-url https://test.pypi.org/legacy/ dist/*
```

### *Zadanie*

```
git checkout task-6
git checkout -b my-solution-6
```
* [ ] Przy użyciu setuptools stwórz paczkę zawierającą core'ową funkcjonalność aplikacji.
    * [ ] zainstaluj paczki `twine` i `build`
    * [ ] utwórz plik `setup.cfg`
    * [ ] w sekcji metadanych umieść:
        - nazwę pakietu
        - swoje imię i nazwisko jako autora
        - swój email
        - krótki opis
        - wersję (używając semantic versioning)
        - licencję (**CC BY-NC-SA 4.0**)
    * [ ] użyj wbudowanego mechanizmu znajdowania pakietów
    * [ ] zbuduj paczkę poleceniem `pyproject-build`
* [ ] Opublikuj ją w [testowym PyPI](https://test.pypi.org) przy uzyciu `twine`, zgodnie z powyższą instrukcją
* [ ] W nowym terminalu, w innym katalogu:
    - [ ] stwórz nowy `virtualenv`
    - [ ] aktywuj `virtualenv`
    - [ ] zainstaluj paczkę - gotowa komenda będzie dostępna na stronie naszej paczki - link można znaleźć na końcu w outpucie twine'a.
    - [ ] sprawdź poprawność instalacji komendą `python -c "import dirwatcher"` jeśli nie będzie błędu - gratulacje!